In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.parallel_processing as pp
import time

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(data):
    time.sleep(1)
    return data * 2

def print_pass(data):
    print(f'{data / 2**5}')
    return data

In [4]:
outputs = pp.queues.ThreadQueue()
class Outer(pp.communicators.Sequential):

    def __init__(self):
        super().__init__()

        self.compute2 = compute
        self.printer = print_pass
        self.aggregator = pp.communicators.Aggregator(20)

        self.registerIOs(
            compute2=pp.queues.ProcessQueue(2),
            printer=pp.queues.TransferQueue('named'),
            aggregator=pp.queues.ThreadQueue(),
            outputs=outputs
        )

caller = Outer()
caller

*-[ProcessQueue(name=None, runner=2)]

   |--(compute2:<function compute at 0x7f94d1999580>)

      |--[TransferQueue(name=named, runner=1)]

         |--(printer:<function print_pass at 0x7f94d19eb740>)

            |--[ThreadQueue(name=None, runner=1)]

               |--(aggregator:Aggregator)

                  |--[ThreadQueue((name=None, runner=1)]-*

In [5]:
with caller:
    for i in range(20):
        caller.put(i)
    
    print(caller.aggregator.wait())

 10%|█         | 2/20 [00:01<00:00, 19.74it/s]

0.0625
0.0


 20%|██        | 4/20 [00:02<00:04,  3.20it/s]

0.125
0.1875


 30%|███       | 6/20 [00:03<00:05,  2.51it/s]

...

0.875
0.9375


 90%|█████████ | 18/20 [00:09<00:00,  2.03it/s]

1.0
1.0625


100%|██████████| 20/20 [00:10<00:00,  2.00it/s]

1.125
1.1875
[2, 0, 4, 6, 10, 8, 12, 14, 16, 18, 20, 22, 26, 24, 28, 30, 32, 34, 36, 38]
